<a href="https://colab.research.google.com/github/avulahemalatha-12/Movie-Recommendation-system/blob/main/MovieRecommendationSystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Step 1: Load the MovieLens Dataset
ratings = pd.read_csv('/content/ratings.csv')
movies = pd.read_csv('/content/movies.csv')

# Step 2: Preprocess the Data
# Create a user-item matrix
user_movie_matrix = ratings.pivot(index='userId', columns='movieId', values='rating').fillna(0)

# Step 3: Implement Collaborative Filtering
# Calculate cosine similarity between users
user_similarity = cosine_similarity(user_movie_matrix)

# Convert to DataFrame for easier handling
user_similarity_df = pd.DataFrame(user_similarity, index=user_movie_matrix.index, columns=user_movie_matrix.index)

# Step 4: Recommend Movies for a New User
# Ask ratings for the new user
new_user_ratings = {
    123: 2,  # Movie ID 123 rated 2
    245: 3,
    345: 5
}

# Create a DataFrame for the new user
new_user_df = pd.DataFrame(new_user_ratings.items(), columns=['movieId', 'rating'])
new_user_df['userId'] = 0  # Assign a user ID for the new user

# Append the new user ratings to the original ratings DataFrame
all_ratings = pd.concat([ratings, new_user_df[['userId', 'movieId', 'rating']]], ignore_index=True)

# Create a new user-item matrix including the new user
new_user_movie_matrix = all_ratings.pivot(index='userId', columns='movieId', values='rating').fillna(0)

# Calculate cosine similarity for the new user
new_user_similarity = cosine_similarity(new_user_movie_matrix)

# Get the similarity scores for the new user
new_user_similarities = new_user_similarity[0]

# Get the indices of the most similar users
similar_users_indices = np.argsort(new_user_similarities)[::-1][1:]
similar_users_indices = similar_users_indices[similar_users_indices < user_movie_matrix.shape[0]]

# Get the ratings of the most similar users using the original user_movie_matrix and the filtered indices
similar_users_ratings = user_movie_matrix.iloc[similar_users_indices]

# Calculate weighted ratings for each movie
weighted_ratings = similar_users_ratings.T.dot(new_user_similarities[similar_users_indices])
recommendations = weighted_ratings / np.array([np.abs(new_user_similarities[similar_users_indices]).sum()])

# Get the top 5 movie recommendations
top_recommendations = recommendations.sort_values(ascending=False).head(5)

# Get movie titles for the recommended movie IDs
recommended_movie_ids = top_recommendations.index
recommended_movies = movies[movies['movieId'].isin(recommended_movie_ids)]

# Display the recommended movies
print("Top 5 Movie Recommendations for the New User:")
print(recommended_movies[['movieId', 'title']])

Top 5 Movie Recommendations for the New User:
      movieId                             title
257       296               Pulp Fiction (1994)
277       318  Shawshank Redemption, The (1994)
314       356               Forrest Gump (1994)
510       593  Silence of the Lambs, The (1991)
1939     2571                Matrix, The (1999)
